In [4]:
import sqlite3
import os

def check_sqlite_db(db_path):
    if not os.path.exists(db_path):
        print(f"❌ Database file not found: {db_path}")
        return

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        print(f"✅ Connected to database: {db_path}")

        # List tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        if not tables:
            print("⚠️ No tables found in the database.")
            return

        print("📋 Tables found in the database:")
        for table in tables:
            table_name = table[0]
            print(f" - {table_name}")

            # Optional: preview first 5 rows from each table
            try:
                cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
                rows = cursor.fetchall()
                print(f"   Sample data from '{table_name}':")
                for row in rows:
                    print(f"     {row}")
            except sqlite3.Error as e:
                print(f"   ⚠️ Could not read from table '{table_name}': {e}")

    except sqlite3.Error as e:
        print(f"❌ SQLite error: {e}")
    finally:
        if 'conn' in locals():
            conn.close()
            print("🔒 Connection closed.")

# Example usage
db_path = '/home/ubuntu/therapy-bro/backend/chat.db'  # Replace with your DB file
check_sqlite_db(db_path)


✅ Connected to database: /home/ubuntu/therapy-bro/backend/chat.db
📋 Tables found in the database:
 - user
   Sample data from 'user':
     (1, 'demo', '$argon2id$v=19$m=65536,t=3,p=4$w/gfgzBmbG0t5ZyTUopxjg$fvLKl0XrikL+N8xShx/z0IIhHbFafFA6MKf6b21HxiI', 'Demo User', None, None, '2025-09-28 11:10:35.273510', None, None, None, 'local')
     (2, 'demov', '$argon2id$v=19$m=65536,t=3,p=4$GAPAuFdqrdXaWyuF8J4TIg$YpwM1kjcYXr2ddN92WHPmlucNFiahdCz8aXIl2ESbi0', 'Demo User', None, None, '2025-09-28 15:17:23.990550', None, None, None, 'local')
     (3, 'abhi.alkane@gmail.com', None, 'Abhishek Sharma', None, None, '2025-09-28 15:44:05.229640', '111847371115025071537', 'abhi.alkane@gmail.com', 'https://lh3.googleusercontent.com/a/ACg8ocLVe7O_odPIktFphm4Mfj9bfl7Zf5uuQuvBG_-1LJPAASjWDv81=s96-c', 'google')
     (4, 'abhishek@textraja.com', None, 'abhishek sharma', None, None, '2025-09-28 20:55:51.542676', '104992090574318543031', 'abhishek@textraja.com', 'https://lh3.googleusercontent.com/a/ACg8ocJ1di6r2T

In [5]:

def get_all_tables(db_path):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
        tables = [row[0] for row in cursor.fetchall()]

        return tables

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
        return []
    finally:
        if conn:
            conn.close()


tables = get_all_tables(db_path)

print("Tables in the database:")
for table in tables:
    print(f" - {table}")


Tables in the database:
 - user
 - chatsession
 - message


In [6]:

def inspect_user_table(db_path):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Check if 'user' table exists
        cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type='table' AND name='user';
        """)
        if not cursor.fetchone():
            print("❌ Table 'user' does not exist.")
            return

        # Get schema (column names and types)
        cursor.execute("PRAGMA table_info(user);")
        schema = cursor.fetchall()
        print("📄 'user' Table Schema:")
        for col in schema:
            cid, name, dtype, notnull, default, pk = col
            print(f" - {name} ({dtype}){' PRIMARY KEY' if pk else ''}")

        # Fetch sample data
        cursor.execute("SELECT * FROM user LIMIT 5;")
        rows = cursor.fetchall()

        if rows:
            print("\n📊 Sample Data:")
            for row in rows:
                print(row)
        else:
            print("\n⚠️ Table 'user' is empty.")

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    finally:
        if conn:
            conn.close()

inspect_user_table(db_path)


📄 'user' Table Schema:
 - id (INTEGER) PRIMARY KEY
 - login_id (VARCHAR)
 - password_hash (VARCHAR)
 - name (VARCHAR)
 - phone (VARCHAR)
 - age (INTEGER)
 - created_at (DATETIME)
 - google_id (VARCHAR)
 - email (VARCHAR)
 - avatar_url (VARCHAR)
 - auth_provider (VARCHAR)

📊 Sample Data:
(1, 'demo', '$argon2id$v=19$m=65536,t=3,p=4$w/gfgzBmbG0t5ZyTUopxjg$fvLKl0XrikL+N8xShx/z0IIhHbFafFA6MKf6b21HxiI', 'Demo User', None, None, '2025-09-28 11:10:35.273510', None, None, None, 'local')
(2, 'demov', '$argon2id$v=19$m=65536,t=3,p=4$GAPAuFdqrdXaWyuF8J4TIg$YpwM1kjcYXr2ddN92WHPmlucNFiahdCz8aXIl2ESbi0', 'Demo User', None, None, '2025-09-28 15:17:23.990550', None, None, None, 'local')
(3, 'abhi.alkane@gmail.com', None, 'Abhishek Sharma', None, None, '2025-09-28 15:44:05.229640', '111847371115025071537', 'abhi.alkane@gmail.com', 'https://lh3.googleusercontent.com/a/ACg8ocLVe7O_odPIktFphm4Mfj9bfl7Zf5uuQuvBG_-1LJPAASjWDv81=s96-c', 'google')
(4, 'abhishek@textraja.com', None, 'abhishek sharma', None, Non